## 1. Tải Dữ Liệu từ CSV

In [2]:
!pip install datasets

In [3]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModel
import numpy as np
import torch
from datasets import load_dataset
import torch.nn as nn
import os
from typing import List
from tqdm import tqdm


# os.environ["CUDA_VISIBLE_DEVICES"] = "1" ## Setup CUDA GPU 1



In [4]:
import torch

# Kiểm tra GPU khả dụng
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU found.")


CUDA available: True
Number of GPUs: 1
GPU 0: Tesla T4


In [5]:
import os
import torch

def select_gpu():
    """
    Kiểm tra GPU khả dụng và tự động chọn GPU phù hợp.
    """
    if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        print(f"Number of GPUs available: {num_gpus}")

        # Duyệt qua các GPU khả dụng để tìm GPU ít sử dụng nhất
        available_gpus = [torch.cuda.get_device_name(i) for i in range(num_gpus)]
        print("Available GPUs:", available_gpus)

        for i in range(num_gpus):
            try:
                # Đặt GPU
                os.environ["CUDA_VISIBLE_DEVICES"] = str(i)
                device = torch.device(f"cuda:{i}")
                torch.cuda.set_device(device)
                print(f"Using GPU: {torch.cuda.get_device_name(device.index)}")
                return device
            except Exception as e:
                print(f"GPU {i} is not suitable: {e}")

        print("No suitable GPU found. Falling back to CPU.")
        return torch.device("cpu")
    else:
        print("No GPUs available. Using CPU.")
        return torch.device("cpu")

# Tự động chọn GPU hoặc CPU
device = select_gpu()

# Kiểm tra lại thiết bị đang sử dụng
print(f"Final selected device: {device}")


Number of GPUs available: 1
Available GPUs: ['Tesla T4']
Using GPU: Tesla T4
Final selected device: cuda:0


In [6]:

class BERTIntentClassification(nn.Module):


    def __init__(self, model_name="bert-base-uncased", num_classes=10, dropout_rate=0.1, cache_dir = "huggingface"):
        super(BERTIntentClassification, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name, cache_dir = cache_dir)
        # Get BERT hidden size
        hidden_size = self.bert.config.hidden_size
        self.ffnn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_classes)
        )


    def freeze_bert(self):
        for param in self.bert.parameters():
            param.requires_grad = False


    def get_pooling(self, hidden_state, attention_mask):
        """
        Get mean pooled representation from BERT hidden states
        Args:
            hidden_state: BERT output containing hidden states
        Returns:
            pooled_output: Mean pooled representation of the sequence
        """
        # Get last hidden state
        last_hidden_state = hidden_state.last_hidden_state  # Shape: [batch_size, seq_len, hidden_size]

        if attention_mask is not None:
            # Expand attention mask to match hidden state dimensions
            attention_mask = attention_mask.unsqueeze(-1)  # [batch_size, seq_len, 1]

            # Mask out padding tokens
            masked_hidden = last_hidden_state * attention_mask

            # Calculate mean (sum / number of actual tokens)
            sum_hidden = torch.sum(masked_hidden, dim=1)  # [batch_size, hidden_size]
            count_tokens = torch.sum(attention_mask, dim=1)  # [batch_size, 1]
            pooled_output = sum_hidden / count_tokens
        else:
            # If no attention mask, simply take mean of all tokens
            pooled_output = torch.mean(last_hidden_state, dim=1)

        return pooled_output


    def forward(self, input_ids, attention_mask, **kwargs):
        """
        Forward pass of the model
        Args:
            input_ids: Input token IDs
            attention_mask: Attention mask for padding
        Returns:
            logits: Raw logits for each class
        """
        # Get BERT hidden states
        hidden_state = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        # Get pooled representation
        hidden_state_pooling = self.get_pooling(hidden_state=hidden_state, attention_mask=attention_mask)

        # Pass through FFNN classifier
        logits = self.ffnn(hidden_state_pooling)

        return logits


In [7]:
class TrainerCustom(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và nhận đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
        logits = outputs

        # Tính toán loss
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss


# 1. Load Dataset and with Dataloader

In [8]:






# # Bước 1: Tải dữ liệu
# # Sử dụng dataset sẵn có từ Hugging Face hoặc tải từ file cục bộ
# dataset = load_dataset("imdb", cache_dir = "huggingface")  # Ví dụ: Dữ liệu IMDB để phân loại sentiment
# # Thay thế trường 'text' thành 'input_ids' trong train_dataset và test_dataset
# def preprocess_dataset(dataset):
#     return dataset.map(lambda example: {
#             "input_ids": example['text'],
#             "label": example['label']
#         },
#         remove_columns=["text"],
#         num_proc=4  # Sử dụng 4 tiến trình song song để xử lý nhanh hơn
#     )

# train_dataset = preprocess_dataset(dataset["train"])
# test_dataset = preprocess_dataset(dataset["test"])



In [9]:
# print(train_dataset)
# # Truy cập mẫu cụ thể
# train_sample = train_dataset[:10]
# test_sample = test_dataset[:2]
# print(train_sample)


# from datasets import Dataset

# train_sample = train_dataset[:10]

# # Chuyển từ dict về Dataset
# train_sample_dataset = Dataset.from_dict(train_sample)
# test_sample_dataset = Dataset.from_dict(test_sample)
# print(train_sample_dataset)
# print(type(train_sample_dataset))
# # Output: <class 'datasets.arrow_dataset.Dataset'>


# # In thử 1 hàng trong test_sample_dataset
# print("First row in test_sample_dataset:")
# print(test_sample_dataset[0])




In [10]:
from datasets import Dataset

def load_csv_dataset(csv_path, text_column, label_column):
    """
    Tải dataset từ file CSV và đổi tên cột.

    Args:
        csv_path (str): Đường dẫn đến file .csv.
        text_column (str): Tên cột chứa văn bản.
        label_column (str): Tên cột chứa nhãn.

    Returns:
        Dataset: Tập dữ liệu đã tải từ file .csv.
    """
    # Tải dữ liệu từ file .csv
    dataset = Dataset.from_csv(csv_path)
    # Đổi tên cột
    dataset = dataset.rename_columns({text_column: "input_ids", label_column: "label"})
    return dataset

# Sử dụng hàm
csv_path = "/content/chatbot_intent_data_v1_En.csv"             # Đường dẫn file CSV
text_column = "input_ids"       # Cột chứa văn bản
label_column = "label"        # Cột chứa nhãn

# Tải dataset
dataset = load_csv_dataset(csv_path, text_column, label_column)

# Kiểm tra dữ liệu
print(dataset)

# Truy cập mẫu cụ thể
sample_dataset = dataset.select(range(10))  # Lấy 10 mẫu đầu tiên
print(sample_dataset)


# In thử 1 hàng trong test_sample_dataset
print("First row in test_sample_dataset:")
print(sample_dataset[0])


Dataset({
    features: ['label', 'input_ids'],
    num_rows: 27
})
Dataset({
    features: ['label', 'input_ids'],
    num_rows: 10
})
First row in test_sample_dataset:
{'label': 'Agree', 'input_ids': 'Yes, I want to show you the picture.'}


In [11]:
def check_invalid_samples(dataset):
    invalid_samples = []
    for idx, sample in enumerate(dataset):
        if not isinstance(sample["input_ids"], str) or sample["input_ids"].strip() == "":
            invalid_samples.append((idx, sample))
    return invalid_samples

# Kiểm tra dữ liệu không hợp lệ
invalid_samples = check_invalid_samples(dataset)
print("\n===== Invalid Samples =====")
print(invalid_samples)



===== Invalid Samples =====
[]


In [12]:
# Tự động phát hiện nhãn và tạo ánh xạ nhãn
def create_label_mapping(dataset_list):
    """
    Tự động phát hiện tất cả các nhãn từ danh sách dataset và ánh xạ chúng thành số nguyên.
    """
    all_labels = set()
    for dataset in dataset_list:
        all_labels.update(dataset["label"])  # Tập hợp tất cả các nhãn từ dataset

    label_to_int = {label: idx for idx, label in enumerate(sorted(all_labels))}
    print(f"Ánh xạ nhãn: {label_to_int}")
    return label_to_int

# Hàm chuyển đổi nhãn
def preprocess_labels(example, label_to_int):
    example["label"] = label_to_int.get(example["label"], -1)  # Gán -1 cho nhãn không hợp lệ
    return example

# Tạo ánh xạ nhãn
label_mapping = create_label_mapping([dataset])

# Áp dụng chuyển đổi nhãn
dataset = dataset.map(lambda example: preprocess_labels(example, label_mapping))

# Kiểm tra kết quả
print(dataset)

# Truy cập mẫu cụ thể
sample_dataset = dataset.select(range(10))  # Lấy 10 mẫu đầu tiên
print(sample_dataset)

# In thử 1 hàng trong sample_dataset
print("First row in sample_dataset:")
print(sample_dataset[0])

Ánh xạ nhãn: {'Agree': 0, 'Decline': 1, 'Fallback': 2, 'Silence': 3, 'Uncertain': 4}
Dataset({
    features: ['label', 'input_ids'],
    num_rows: 27
})
Dataset({
    features: ['label', 'input_ids'],
    num_rows: 10
})
First row in sample_dataset:
{'label': 0, 'input_ids': 'Yes, I want to show you the picture.'}


In [13]:
def split_dataset(dataset, test_size=0.2, seed=42):
    """
    Chia dataset thành tập train và test.

    Args:
        dataset (Dataset): Tập dữ liệu đầy đủ.
        test_size (float): Tỷ lệ dữ liệu test (0.0 - 1.0).
        seed (int): Seed để chia dữ liệu ngẫu nhiên.

    Returns:
        tuple: (train_dataset, test_dataset) - Tập train và test.
    """
    if not (0.0 < test_size < 1.0):
        raise ValueError("test_size phải nằm trong khoảng (0.0, 1.0)")
    if len(dataset) < 2:
        raise ValueError("Dataset phải có ít nhất 2 mẫu để chia.")

    train_test_split = dataset.train_test_split(test_size=test_size, seed=seed)
    print(f"Chia dataset: {len(train_test_split['train'])} mẫu train, {len(train_test_split['test'])} mẫu test")
    return train_test_split["train"], train_test_split["test"]

# Chia dataset
train_dataset, test_dataset = split_dataset(dataset, test_size=0.3)

# Kiểm tra dữ liệu
print("Train dataset:", train_dataset)
print("Test dataset:", test_dataset)

# Truy cập mẫu cụ thể
sample_train_dataset = train_dataset.select(range(8))  # Lấy 10 mẫu đầu tiên từ train
sample_test_dataset = test_dataset.select(range(9))    # Lấy 10 mẫu đầu tiên từ test

print("Sample train dataset:", sample_train_dataset)
print("Sample test dataset:", sample_test_dataset)

Chia dataset: 18 mẫu train, 9 mẫu test
Train dataset: Dataset({
    features: ['label', 'input_ids'],
    num_rows: 18
})
Test dataset: Dataset({
    features: ['label', 'input_ids'],
    num_rows: 9
})
Sample train dataset: Dataset({
    features: ['label', 'input_ids'],
    num_rows: 8
})
Sample test dataset: Dataset({
    features: ['label', 'input_ids'],
    num_rows: 9
})


# 2. Tokenizer

In [14]:





# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir = "huggingface")
model = BERTIntentClassification(
    model_name=model_name,
    num_classes=5
)
model.freeze_bert() # Froze Layer BERT
max_seq_length = 512


def collate_fn(features):
    inputs = []
    labels = []
    for element in features:
        inputs.append(element.get("input_ids"))
        labels.append(element.get("label"))

    labels = torch.tensor(labels, dtype=torch.long)

    token_inputs = tokenizer(
        inputs,
        add_special_tokens=True,
        truncation=True,
        padding=True,
        max_length=max_seq_length,
        return_overflowing_tokens=False,
        return_length=False,
        return_tensors="pt",
    )
    token_inputs.update({
        "labels": labels,
    })
    return token_inputs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# 3. Train Model

## 3.1 Log Wandb

In [15]:
!pip install --upgrade wandb

In [16]:
!pip install python-dotenv

In [17]:
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()

# Lấy key từ biến môi trường
wandb_api_key = os.getenv("WANDB_API_KEY")
print(wandb_api_key[:5])

c8767


In [18]:
import wandb
import os

# Lấy API key từ biến môi trường và đăng nhập
wandb.login(key=os.getenv("WANDB_API_KEY"))


wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Cách thiết lập thông qua TrainingArguments
Khi sử dụng Trainer, bạn có thể đặt tên dự án trực tiếp trong TrainingArguments bằng cách sử dụng tham số report_to và run_name. Tuy nhiên, để đặt project, bạn cần khởi tạo một phiên wandb trước hoặc truyền cấu hình này thông qua wandb.init().

Điều chỉnh TrainingArguments:
```python
training_args = TrainingArguments(
    output_dir="./results_",          # Thư mục lưu kết quả
    eval_strategy="epoch",           # Đánh giá sau mỗi epoch
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",            # Thư mục lưu log
    logging_strategy="steps",        # Log theo steps
    logging_steps=10,                # Log sau mỗi 10 bước
    save_strategy="epoch",           # Lưu checkpoint sau mỗi epoch
    save_total_limit=3,              # Lưu tối đa 3 checkpoint
    report_to="wandb",               # Báo cáo log tới wandb
    run_name="bert_run_1"            # Tên phiên chạy trên wandb
)
```

## 3.2 Train

### Ver 1.2.3

Dưới đây là bảng tóm tắt chi tiết cách lưu mô hình dựa trên chiến lược được đề xuất:

| **Loại Model**    | **Điều Kiện Lưu**                                                                 | **Thư Mục Lưu Trên Local**       | **Số Lượng Lưu Trên Local**        | **Thông Tin Thêm**                              | **Đồng Bộ Lên WandB**                  |
|--------------------|-----------------------------------------------------------------------------------|-----------------------------------|------------------------------------|-----------------------------------------------|-----------------------------------------|
| **Best Model**     | Khi `eval_loss` giảm                                                             | `output_dir/best_model`           | Chỉ lưu một bản duy nhất           | Lưu thông tin `epoch` và `eval_loss`.          | Có: Artifact `best_model`. Thêm `epoch` và `loss` vào `metadata`. |
| **Final Checkpoint** | Sau mỗi epoch (checkpoint cuối của epoch)                                        | `output_dir/checkpoint-epoch-<n>` | Tối đa 3 checkpoint gần nhất       | Không có thông tin đặc biệt.                   | Không đồng bộ (tránh trùng lặp dữ liệu lớn). |
| **Custom Checkpoint** (tùy chọn) | Sau một số bước cố định hoặc mốc quan trọng (nếu cần thiết, ví dụ: mỗi 5 epoch) | Tùy chỉnh, ví dụ: `output_dir/checkpoint-step-<n>` | Theo ý muốn, hoặc không giới hạn | Thêm các mốc quan trọng để phân tích sau này. | Tùy chọn (không bắt buộc).              |

---

### **Chi tiết về bảng**
1. **Best Model**:
   - Điều kiện: `eval_loss` giảm.
   - Chỉ lưu một phiên bản tốt nhất.
   - Lưu thông tin epoch và loss để dễ dàng tham khảo hoặc tải xuống sau này.

2. **Final Checkpoint**:
   - Được lưu sau mỗi epoch.
   - Giới hạn số lượng checkpoint lưu trên local để tiết kiệm bộ nhớ (ví dụ: tối đa 3 checkpoint).
   - Không lưu thông tin thêm vào checkpoint.

3. **Custom Checkpoint** (tùy chọn):
   - Có thể sử dụng nếu bạn muốn lưu checkpoint tại các mốc thời gian cụ thể, chẳng hạn như mỗi 5 epoch hoặc sau một số bước huấn luyện (steps).
   - Thích hợp khi bạn cần kiểm tra tiến độ huấn luyện chi tiết hơn hoặc muốn lưu backup.

---

### **Tóm tắt logic**
- **Best Model**:
  - Lưu vào thư mục cố định (`best_model`).
  - Ghi đè khi có `eval_loss` mới tốt hơn.
  - Đồng bộ lên WandB.

- **Final Checkpoint**:
  - Lưu sau mỗi epoch.
  - Xóa checkpoint cũ nhất nếu vượt giới hạn `save_total_limit`.
  - Không đồng bộ lên WandB (tránh lãng phí không gian lưu trữ).

- **Custom Checkpoint**:
  - Tùy chọn nếu bạn cần lưu thêm để phục vụ các mục đích cụ thể.

Nếu bạn cần thêm bất kỳ chi tiết nào khác, hãy cho mình biết nhé! 😊

### **Bảng Tóm Tắt: Lưu Best Model và Last Model**

| **Loại Model**    | **Khi Nào Cần Lưu**                                                                                         | **Ưu Điểm**                                                                                       | **Hạn Chế**                                                                                      |
|--------------------|------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------|
| **Best Model**     | - Khi muốn triển khai mô hình tốt nhất với `eval_loss` thấp nhất hoặc `accuracy` cao nhất.                   | - Đảm bảo lưu lại mô hình có hiệu suất tốt nhất trên tập validation.<br>- Phù hợp để triển khai.   | - Không lưu trạng thái đầy đủ (optimizer, scheduler).<br>- Không tiếp tục huấn luyện từ trạng thái này. |
| **Last Model**     | - Khi cần tiếp tục huấn luyện (fine-tuning) hoặc khôi phục trạng thái sau khi huấn luyện kết thúc.         | - Lưu đầy đủ trạng thái (weights, optimizer, scheduler).<br>- Phù hợp để tiếp tục huấn luyện.    | - Có thể không phải là mô hình tốt nhất (do overfitting hoặc underfitting).                     |
| **Chỉ Lưu Best**   | - Khi chỉ quan tâm đến triển khai mô hình tốt nhất, không cần tiếp tục huấn luyện sau này.                  | - Tiết kiệm tài nguyên lưu trữ.<br>- Tập trung vào mô hình tối ưu cho triển khai.                | - Không thể tiếp tục huấn luyện nếu cần.                                                         |
| **Chỉ Lưu Last**   | - Khi muốn đảm bảo khả năng khôi phục trạng thái để tiếp tục huấn luyện.                                    | - Khôi phục hoàn toàn quá trình huấn luyện.<br>- Phù hợp cho fine-tuning hoặc thử nghiệm sau này. | - Không đảm bảo đây là mô hình tốt nhất để triển khai.                                           |
| **Lưu Cả Hai**     | - Khi cần cả triển khai mô hình tốt nhất và tiếp tục huấn luyện sau này.                                    | - Kết hợp ưu điểm của cả Best Model và Last Model.<br>- Linh hoạt trong sử dụng.                 | - Tốn thêm tài nguyên lưu trữ và thời gian.                                                     |

---

### **Chiến Lược Tối Ưu**
| **Loại Lưu** | **Tần Suất**                          | **Chiến Lược**                                                                                             |
|--------------|---------------------------------------|-----------------------------------------------------------------------------------------------------------|
| **Best Model** | Khi `eval_loss` giảm                 | Lưu mỗi lần `eval_loss` giảm để đảm bảo mô hình tốt nhất luôn được lưu.                                    |
| **Last Model** | Sau khi huấn luyện kết thúc          | Lưu trạng thái cuối cùng của quá trình huấn luyện (weights + optimizer + scheduler).                      |
| **Kết hợp**   | Best Model: Mỗi khi `eval_loss` giảm<br>Last Model: Sau khi kết thúc | Lưu cả Best Model để triển khai và Last Model để tiếp tục huấn luyện khi cần thiết.                      |

---

### **Lựa Chọn Phù Hợp**
- **Dự án triển khai mô hình nhanh**: Lưu **Best Model**.
- **Dự án nghiên cứu hoặc fine-tuning tiếp**: Lưu **Last Model**.
- **Dự án quy mô lớn, cần cả triển khai và mở rộng**: Lưu **cả hai**.

Hãy chọn chiến lược lưu phù hợp với mục tiêu dự án của bạn! 🚀

Thui, ko lưu local nữa, lưu tất trên wandb đi.
- Với best model: lưu lên wandb khi loss giảm và đã sau 10 epochs  
(Lưu Best Model ngay khi eval_loss giảm ở local, sau 10 epochs thì đồng bộ cái best lên wandb, sau đó xoá các file best ở local).
Chỉ đồng bộ lên WandB mỗi 10 epochs.)
- Với last model: lưu lên wandb sau mỗi 10 epochs. (lưu local trước -> đồng bộ lên wandb sẽ xoá file local)
+, Trong quá trình lưu thì việc training vẫn diễn ra Parallel

đều lưu đầy đủ toàn bộ tham số để có thể train thêm từ cả ở best model và last model

In [19]:
# import os
# os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [20]:
# class TrainerCustom(Trainer):

#     def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
#         """
#         How the loss is computed by Trainer. By default, all models return the loss in the first element.

#         Subclass and override for custom behavior.
#         """
#         if "labels" in inputs:
#             labels = inputs.pop("labels")
#         else:
#             labels = None

#         # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
#         cross_entropy_loss = nn.CrossEntropyLoss()

#         # Chạy mô hình và nhận đầu ra (logits)
#         outputs = model(**inputs)

#         # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
#         logits = outputs

#         if labels is None:
#             print("Labels are None during compute_loss.")
#         if logits is None:
#             print("Logits are None during compute_loss.")

#         # Tính toán loss
#         loss = cross_entropy_loss(logits, labels)

#         # Trả về loss và outputs nếu cần
#         return (loss, outputs) if return_outputs else loss


In [21]:

# import wandb

# # Khởi tạo wandb
# wandb.init(
#     project="bert-intent-classification",  # Tên dự án
#     name="bert_run_3"                     # Tên phiên chạy
# )


# # Bước 6: Cài đặt tham số huấn luyện
# training_args = TrainingArguments(
#     output_dir="./result__s",          # Thư mục lưu kết quả
#     eval_strategy="epoch",    # Đánh giá sau mỗi epoch
#     learning_rate=2e-4,
#     per_device_train_batch_size=128,
#     per_device_eval_batch_size=128,
#     num_train_epochs=50,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_strategy="steps",
#     logging_steps=1,  # Ghi logs mỗi 500 bước huấn luyện
#     save_strategy="no",          # Lưu trọng số sau mỗi epoch
#     save_total_limit=3,
#     label_names = ["labels"],
#     report_to="wandb",
#     run_name="bert_run_3"
# )


# batch = collate_fn([sample_test_dataset[0]]) # Tạo một batch từ một mẫu đơn lẻ (sample_test_dataset[0]) để kiểm tra xem hàm collate_fn có hoạt động đúng không.
# print(batch)

# # metrics = trainer.evaluate()
# # Mục đích: Chạy giai đoạn evaluation (đánh giá) trên eval_dataset (sample_test_dataset) và tính toán các metrics như:
# trainer = TrainerCustom(
#     model=model,
#     args=training_args,
#     train_dataset=sample_train_dataset,
#     eval_dataset=sample_test_dataset,
#     tokenizer=tokenizer,
#     data_collator=collate_fn,
# )

# metrics = trainer.evaluate()
# print(metrics)  # Kiểm tra xem có "eval_loss" hay không


In [22]:

# # Bước 7: Tạo Trainer
# trainer = TrainerCustom(
#     model=model,
#     args=training_args,
#     train_dataset=sample_train_dataset,
#     eval_dataset=sample_test_dataset,
#     tokenizer=tokenizer,
#     data_collator = collate_fn,
# )

# # Bước 8: Huấn luyện
# trainer.train()

# # Kết thúc phiên wandb
# wandb.finish()




In [31]:
from concurrent.futures import ThreadPoolExecutor
import wandb
import os
import shutil
import time

class TrainerCustom(Trainer):
    def __init__(self, *args, save_every_n_epochs=10, **kwargs):
        super().__init__(*args, **kwargs)
        if torch.cuda.is_available():
            print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
        else:
            print("Trainer is running on CPU.")

        self.best_eval_loss = float("inf")  # Giá trị loss tốt nhất ban đầu
        self.save_every_n_epochs = save_every_n_epochs  # Tần suất lưu lên WandB
        self.best_model_info = {"epoch": None, "loss": None}
        self.last_saved_epoch = 0  # Epoch cuối cùng đã lưu Best Model và Last Model
        self.executor = ThreadPoolExecutor(max_workers=3)  # Cho phép tối đa 2 luồng song song

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """

        # Kiểm tra thiết bị của mô hình và dữ liệu
        print("Model device:", next(model.parameters()).device)
        print("Input device:", inputs["input_ids"].device)
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và nhận đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
        logits = outputs

        if labels is None:
            print("Labels are None during compute_loss.")
        if logits is None:
            print("Logits are None during compute_loss.")

        # Tính toán loss
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss

    def async_save_model(self, model_dir, artifact_name, metadata=None):
        """
        Lưu mô hình vào local và đồng bộ lên WandB trong luồng song song.
        """
        def save():
            start_time = time.time()
            try:
                # Xóa tất cả các thư mục tmp_best_model_ trước đó
                for folder in os.listdir("."):
                    if folder.startswith("tmp_best_model_epoch_") and folder != model_dir:
                        shutil.rmtree(folder, ignore_errors=True)
                        print(f"Removed old temporary directory: {folder}")

                # Lưu mô hình vào thư mục tạm
                self.save_model(model_dir)

                # Đồng bộ lên WandB
                artifact = wandb.Artifact(artifact_name, type="model")
                artifact.add_dir(model_dir)
                if metadata:
                    artifact.metadata = metadata
                wandb.log_artifact(artifact)
            except Exception as e:
                print(f"Error during saving or syncing model {artifact_name}: {e}")
            finally:
                # Xóa thư mục tạm hiện tại sau khi đồng bộ
                try:
                    shutil.rmtree(model_dir, ignore_errors=True)
                    print(f"Successfully removed temporary directory: {model_dir}")
                except Exception as e:
                    print(f"Error removing temporary directory {model_dir}: {e}")

            elapsed_time = time.time() - start_time
            print(f"Model saved and uploaded to WandB: {artifact_name} in {elapsed_time:.2f} seconds")

        self.executor.submit(save)




    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        eval_loss = metrics.get("eval_loss")

        # Cập nhật Best Model nếu eval_loss giảm
        # Lưu Best Model ngay khi eval_loss giảm (local).
        # Chỉ đồng bộ lên WandB mỗi 10 epochs.

        if eval_loss is not None and eval_loss < self.best_eval_loss:
            print(f"New best eval_loss: {eval_loss}")
            self.best_eval_loss = eval_loss
            self.best_model_info = {"epoch": self.state.epoch, "loss": eval_loss}

            # Log thông tin Best Model lên WandB
            wandb.log({
                "best_eval_loss": self.best_eval_loss,
                "best_model_epoch": self.best_model_info.get("epoch", -1)
            })

            # Lưu Best Model vào thư mục tạm (local)
            best_model_dir = f"./tmp_best_model_epoch_{int(self.state.epoch)}"
            self.save_model(best_model_dir)

            # Đồng bộ lên WandB mỗi 10 epochs
            if int(self.state.epoch) % self.save_every_n_epochs == 0:
                artifact_name = f"best_model_epoch_{int(self.state.epoch)}"
                self.async_save_model(best_model_dir, artifact_name, self.best_model_info)

        return metrics

    def save_last_model(self):
        """
        Lưu Last Model lên WandB sau mỗi N epochs.
        """
        if int(self.state.epoch) % self.save_every_n_epochs == 0 and int(self.state.epoch) != self.last_saved_epoch:
            print(f"Saving Last Model at epoch {self.state.epoch} to WandB...")
            last_model_dir = f"./tmp_last_model_epoch_{int(self.state.epoch)}"
            artifact_name = f"last_model_epoch_{int(self.state.epoch)}"
            self.async_save_model(last_model_dir, artifact_name)

            # Log thông tin Last Model lên WandB
            wandb.log({
                "last_model_epoch": self.state.epoch
            })

            # Cập nhật epoch cuối cùng đã lưu
            self.last_saved_epoch = int(self.state.epoch)

    def train(self, *args, **kwargs):
        result = super().train(*args, **kwargs)

        # Sau mỗi epoch, lưu Last Model lên WandB
        self.save_last_model()
        # Chờ tất cả các luồng lưu hoàn thành trước khi kết thúc
        self.executor.shutdown(wait=True)

        return result


# Bước 6: Cài đặt tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./result__s",          # Thư mục lưu kết quả
    eval_strategy="epoch",    # Đánh giá sau mỗi epoch
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=1,  # Ghi logs mỗi 500 bước huấn luyện
    save_strategy="no",          # Lưu trọng số sau mỗi epoch
    save_total_limit=3,
    label_names = ["labels"],
    report_to="wandb",
    run_name="bert_run_3"
)


import wandb

# Khởi tạo wandb
wandb.init(
    project="bert-intent-classification",  # Tên dự án
    name="bert_run_3",                     # Tên phiên chạy
    config={"gpu": torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else "CPU"}
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
if torch.cuda.is_available():
    print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("Trainer is running on CPU.")

trainer = TrainerCustom(
    model=model,
    args=training_args,
    train_dataset=sample_train_dataset,
    eval_dataset=sample_test_dataset,
    data_collator=collate_fn,
    save_every_n_epochs=10  # Lưu Best Model và Last Model mỗi 10 epochs
)

trainer.train()
wandb.finish()


Trainer is running on GPU: Tesla T4
Trainer is running on GPU: Tesla T4
Model device: cuda:0
Input device: cuda:0


Epoch,Training Loss,Validation Loss
1,0.000000,3.504333
2,0.000000,3.508570
3,0.000000,3.512193
4,0.000000,3.514808
5,0.000000,3.518519
6,0.000000,3.519195
7,0.000000,3.520812
8,0.000000,3.521818
9,0.000000,3.524044
10,0.000000,3.525590


Model device: cuda:0
Input device: cuda:0
New best eval_loss: 3.5043325424194336
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cuda:0
Model device: cuda:0
Input device: cu

wandb: Adding directory to artifact (./tmp_last_model_epoch_50)... Done. 5.4s


Successfully removed temporary directory: ./tmp_last_model_epoch_50
Model saved and uploaded to WandB: last_model_epoch_50 in 13.87 seconds


best_eval_loss,▁
best_model_epoch,▁
eval/loss,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██████
eval/runtime,▂▁▁▁▁▂▁▁▁▁▁▁▁▁▂▃▄▄▁▁▁▄▄▂▁▁▁▂▁█▆▂▃▄▁▁▂▂▂▂
eval/samples_per_second,▅▇▇▇▇▄▆▇▇▆▇▇▇▆▄▃▃▇█▇▃▃▅▇▆█▆█▁▃▅▃▃▄▇▇▆▆▆▆
eval/steps_per_second,▅▇▇▇▇▄▇▆▇▇▇▇▇▇▆▄▃▃▇██▃▃▅▆▇█▆▇▁▂▅▃▃▃█▆▆▆▆
last_model_epoch,▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▁▁▁▂▂▃▃▃▂▂▂▃▂▃▄█▃▂▄▅▄▂▄▅▃▅▁▃▃▅▃▄▂▂▄▃▃▂▃▁
train/learning_rate,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁


In [24]:
# Bước 9: Đánh giá trên tập kiểm tra
trainer.evaluate()

Error: You must call wandb.init() before wandb.log()

# Inference

In [ ]:
sentence = "What is the weather like today?"


inputs = tokenizer(
    sentence,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=512
)



In [ ]:
model.eval()  # Đặt mô hình ở chế độ đánh giá (không tính gradient)
with torch.no_grad():  # Không cần tính gradient
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()  # Lấy nhãn dự đoán
    print(f"Predicted class: {predicted_class}")
